### README
This notebook is to be ran whenever there is an update to the module mappings on myedurec

Some manual work will need to be done to check if any new universities have been added, and we will have to update
the country/continent those universities are in, as well as cross-reference with existing universities to see if they
are the same entity

In [19]:
import pandas as pd
import numpy as np
import json, requests
import time
import glob
import re
import copy
import pycountry_convert as pc

In [20]:
# This fname is the csv file downloaded from myedurec
fname = "myedurec_retrieved_160721.csv"

In [21]:
# Initialize df
# This encoding is required because there are some non-ASCII characters
df = pd.read_csv(fname, encoding="iso-8859-1")

In [22]:
def proper_name(string):
    if type(string) != str:
        return string
    else:
        return string.replace('???',"'")

def clean_data(original_data):
    data = original_data
    not_nan = lambda x: x is not np.nan
    data = data[data['Partner University'].map(not_nan)]
    data = data[data['PU Module 1 Title'].map(not_nan) | data['PU Module 2 Title'].map(not_nan)]
    data = data[data['NUS Module 1 Title'].map(not_nan) | data['NUS Module 2 Title'].map(not_nan)]
    data = data[data['NUS Module 1'].map(not_nan) | data['NUS Module 2'].map(not_nan)]
    data['PU Module 1 Title'] = data['PU Module 1 Title'].map(proper_name)
    data['PU Module 2 Title'] = data['PU Module 2 Title'].map(proper_name)
    data['NUS Module 1 Title'] = data['NUS Module 1 Title'].map(proper_name)
    data['NUS Module 2 Title'] = data['NUS Module 2 Title'].map(proper_name)
    return data

In [23]:
cleaned_data = clean_data(df)
cleaned_data.to_csv("cleaned_mappings.csv",index=False)
cleaned_data.head()

,Faculty,Partner University,PU Module 1,PU Module 1 Title,PU Mod1 Credits,PU Module 2,PU Module 2 Title,PU Mod2 Credits,NUS Module 1,NUS Module 1 Title,NUS Mod1 Credits,NUS Module 2,NUS Module 2 Title,NUS Mod2 Credits,Pre Approved?
0,Faculty of Arts & Social Sci,The Hong Kong Polytechnic University,CBS241,Elementary Chinese II (for Non-Chinese speakin...,1.0,NaN,NaN,NaN,LAC2731,Department Exchange Module,3.0,NaN,NaN,NaN,Y
1,Faculty of Arts & Social Sci,The Hong Kong Polytechnic University,CC2C08,Mutual Impressions of China and the West,3.0,NaN,NaN,NaN,PS2238,Int'l Politics of NE Asia,4.0,NaN,NaN,NaN,Y
2,Faculty of Arts & Social Sci,Hong Kong University of Science & Technology,LANG1120,Chinese for Non-Chinese Language Background St...,1.0,NaN,NaN,NaN,LAC1731,Department exchange module,3.0,NaN,NaN,NaN,Y
3,Faculty of Arts & Social Sci,City University of Hong Kong,AIS3126,International Political Economy,3.0,NaN,NaN,NaN,PS3238,Int'l Political Economy,4.0,NaN,NaN,NaN,Y
4,Faculty of Arts & Social Sci,City University of Hong Kong,GE2210,China: A Socio-Political Transformation,3.0,NaN,NaN,NaN,PS2248,Chinese Politics,4.0,NaN,NaN,NaN,Y


### READ THIS BEFORE RUNNING THE FOLLOWING CHUNK OF CODE!
Rename the old 'school_country_continent_mapping.csv' to 'OLD.csv' before running!

This block of code has some work done manually since there is no easy way to check
which country each school is in. We will have to add the countries in manually if 
we miss out the first step!

In [24]:
# df1 = pd.read_csv('OLD.csv')

# countries = set(df1['Country'])
# full_continent_name = {
#     'AF': 'Africa',
#     'AS': 'Asia',
#     'EU': 'Europe',
#     'NA': 'North America',
#     'OC': 'Oceania',
#     'SA': 'South America'
# }

# for i in range(len(df1)):
#     if df1['Country'][i] != 'UNKNOWN':
#         c = pc.country_name_to_country_alpha2(df1['Country'][i], cn_name_format="default")
#         continent = pc.country_alpha2_to_continent_code(c)
#         df1['Continent'][i] = full_continent_name[continent]
#     else:
#         df1['Continent'][i] = 'Europe'

# df1.to_csv('school_country_continent_mapping.csv',index=False)
# del df1

### Note:
This list is created manually. I could not think of any fast/efficient way to determine if 2 schools
are the same entity

In [25]:
mappings = pd.read_csv("cleaned_mappings.csv")

modified_mappings = mappings.copy()

equivalent_schools_mapping = {
    'Aalto University': ['Aalto University, Helsinki'],
    'Aarhus School of Business': ['Aarhus University'],
    'Cornell University': ['Cornell Univ Coll of Agriculture & Life Sciences', 'Cornell Univ Coll of Human Ecology'],
    'Georgetown University': ['Georgetown University Law Center','Georgetown University, Washington D.C.'],
    'Humboldt University of Berlin': ['Humboldt-Universitaet zu Berlin'],
    'Imperial College London': ['Imperial College Business School'],
    'Leiden University': ['Leiden University Medical Center (LUMC)'],
    'University College London': ['University College London, University of London'],
    'Universite Catholique De Louvain': ['Universite Catholique de Louvain'],
}

for school in equivalent_schools_mapping:
    for equiv in equivalent_schools_mapping[school]:
        tmp_df = modified_mappings[modified_mappings['Partner University'] == equiv]
        modified_mappings['Partner University'].iloc[tmp_df.index] = [school] * len(tmp_df)

UCs_df = modified_mappings[modified_mappings['Partner University'] == 'University of California']
for school in modified_mappings['Partner University'].unique():
    if 'University of California' in school and school != 'University of California':
        to_append = UCs_df.copy()
        to_append['Partner University'] = [school] * len(to_append)
        modified_mappings = modified_mappings.append(to_append)
modified_mappings.index = range(len(modified_mappings))

school_country_continent_mapping = pd.read_csv('school_country_continent_mapping.csv')
country, continent = [], []
for i in range(len(modified_mappings)):
    school = modified_mappings['Partner University'][i]
    df = school_country_continent_mapping[school_country_continent_mapping['Partner University'] == school]
    try:
        country.append(df['Country'].iloc[0])
        continent.append(df['Continent'].iloc[0])
    except:
        print(school)
modified_mappings['Country'] = country
modified_mappings['Continent'] = continent

modified_mappings.to_csv('cleaned_mappings_with_locations.csv',index=False)

This following cell is to download the data from the NUSMods api

Only run this to update current data

In [26]:
# import httplib2
# from bs4 import BeautifulSoup, SoupStrainer

# http = httplib2.Http()
# url = "https://nusmods.com/api/v2/2020-2021/"
# status, response = http.request(url + "modules/")

# links = []
# for link in BeautifulSoup(response, parse_only=SoupStrainer('a')):
#     if link.has_attr('href') and link['href'][-5:] == '.json':
#         links.append(link['href'])
        

# start = time.time()
# for i, link in enumerate(links):
#     if (i % 100 == 0):
#         print(str(i+1) + '/' + str(len(links)))
#     moduleDetails = requests.get(url + f"modules/{link}").json()
#     with open("module_details/"+f"{link}", "w") as f:
#         json.dump(moduleDetails, f)
    
# end = time.time()
# print("Time taken: " + str(round(end-start)) + " seconds")

In [27]:
def load_data():
    arr = []
    for fname in sorted(glob.glob('module_details/*.json')):
        with open(fname,'r') as f:
            data = json.load(f)
            arr.append(data)
    return arr
jsons = load_data()
len(jsons)

12759

In [28]:
def get_attributes(jsons):
    attrs = set()
    for data in jsons:
        for attribute in data:
            attrs.add(attribute)
    return attrs
attributes = get_attributes(jsons)

In [29]:
def join_preclusions(jsons):
    preclusions_mapping = {}
    for data in jsons:
        if 'preclusion' in data:
            preclusions_mapping[data['moduleCode']] = data['preclusion']
        else:
            preclusions_mapping[data['moduleCode']] = ''
    def preclusions_string_to_set(preclusion_string):
        # Get all alphaNumeric substrings
        alpha_numerics = re.split('[^a-zA-Z0-9]', preclusion_string)

        def is_module_code(string):
            right_length = len(string) >= 4
            all_caps = string == string.upper()
            contains_digits = bool(re.search(r'\d', string))
            return right_length and all_caps and contains_digits

        return set(filter(is_module_code, alpha_numerics))
    for preclusion in preclusions_mapping:
        preclusions_mapping[preclusion] = preclusions_string_to_set(preclusions_mapping[preclusion])
    to_delete = []
    for preclusion in preclusions_mapping:
            preclusions_mapping[preclusion] = list(preclusions_mapping[preclusion])
    return preclusions_mapping
preclusions_mapping = join_preclusions(jsons)

In [30]:
with open('preclusionMappings.json', 'w') as f:
    json.dump(preclusions_mapping, f)

In [31]:
def get_equivalent_modules(jsons):
    equivalent_modules = join_preclusions(jsons)
    
    module_titles = {}
    for json in jsons:
        title = json['title'].upper()
        if title not in module_titles:
            module_titles[title] = []
        module_titles[title].append(json['moduleCode'])

    equivalent_modules = copy.deepcopy(preclusions_mapping)
    for module_title in module_titles:
        equivalents = module_titles[module_title]
        for module_code in equivalents:
            for equiv in equivalents:
                if equiv not in equivalent_modules[module_code]:
                    equivalent_modules[module_code].append(equiv)
    return equivalent_modules

In [32]:
equivalent_modules = get_equivalent_modules(jsons)
with open('equivalentModuleMappings.json', 'w') as f:
    json.dump(equivalent_modules, f)

In [33]:
modules = []
for j in jsons:
    modules.append(f"{j['moduleCode']} {j['title']}")
modules.sort()

with open('../src/data/modules.json', 'w') as f:
    json.dump(modules,f)

countries = sorted(modified_mappings['Country'].unique())
with open('../src/data/countries.json', 'w') as f:
    json.dump(countries, f)
continents = sorted(modified_mappings['Continent'].unique())
with open('../src/data/continents.json', 'w') as f:
    json.dump(continents, f)
schools = sorted(modified_mappings['Partner University'].unique())
with open('../src/data/schools.json', 'w') as f:
    json.dump(schools, f)

In [34]:
s0 = set(modified_mappings['NUS Module 1'].dropna().unique())
s1 = set(modified_mappings['NUS Module 2'].dropna().unique())
mapped_modules = s0.union(s1)
reduced_module_list = set()
for module in list(mapped_modules):
    if module in equivalent_modules:
        reduced_module_list.add(module)
        reduced_module_list.update(equivalent_modules[module])
        
# There is a module code "2002" in this list, which must be a bug, hence we removed it
reduced_module_list = sorted(list(reduced_module_list))[1:]

reduced_modules = []
for mod in reduced_module_list:
    try:
        with open(f"module_details/{mod}.json", 'r') as f:
                data = json.load(f)
                reduced_modules.append(f"{data['moduleCode']} {data['title']}")
    except:
        print(mod)
reduced_modules.sort()

with open('../src/data/reducedModules.json', 'w') as f:
    json.dump(reduced_modules,f)

AY2001
AY2018
BH2413
BH3101
BH3102
BH3103
BH3113
BH3115
BH3201
BH3202
BH3214
BH3301
BH3313A
BH3420
BK1006
BK2001
BK2006
BK3100
BK3101
BK3102
BK3105
BK3108
BK3202
BK3203
BK3206
BK3305
BK3309M
BK3505
BSP3001E
BZ1004
BZ2003
BZ3102
BZ3105
BZ3301
BZ3302
BZ3303
BZ3304
BZ3402
BZ3414
BZ3501
BZ3503
BZ3605
BZ3614
CE1402
CF3101
CM1161
CM1503
CM2263
CM2264
CM3262
CM3265
CM3266
CM4221
CN3411
CP3101B
CS1010FC
CS1101
CS1101C
CS1104
CS2102S
CS2305S
CS3213
CS3246
CS3252
CS3253
CS3254
CS4259
CS4260
CS4267
CS4342
CS5212
CS5262
DOS3711
DSC3213
DSC3221
DSN3701
EC2231
EE1401
EE1461
EE2004
EE3002
EE3204
EE3406
EG1401
EG1402
EG2401
ESP2104
EU3228
GEK1009
GEK1511
GEK1542
GEM2010K
IC2231
LSM1101
LSM1103
LSM3213
LSM3221
LSM3257
LSM3271
LSM4233
MA1101
MA1102
MA1505C
MA2101H
MA2118
MA2118H
MA2201
MA2203
MA2205
MA2215
MA3215
MA3235
MA3245
MA3250
ME4234
ME4273
MKT2413A
MKT2413B
MKT4415C
MLE5201
MQ2101
MQ2201
MQ2203
MQ2204
MQ3201
MQ3202
PC1221FC
PC1431FC
PC2174
PC2231
PF2501
PS2209
PS2209B
PS3203B
PS3880B
QF2101
RE38

### WARNING
There are modules that can be mapped but are missing from the NUSMods API data. Almost all of these 
modules are dummy codes, so we will ignore this

In [35]:
s0 = set(modified_mappings['NUS Module 1'].dropna().unique())
s1 = set(modified_mappings['NUS Module 2'].dropna().unique())
s = s0.union(s1)
S = set()
not_inside = []
for mod in s:
    try:
        S.add(mod)
        S.update(set(equivalent_modules[mod]))
    except:
        not_inside.append(mod)
not_inside.sort()

vals = [(mod0 in not_inside or mod1 in not_inside) for mod0, mod1 in zip(modified_mappings['NUS Module 1'], modified_mappings['NUS Module 2'])]
modified_mappings[vals]['NUS Module 1 Title'].unique()

array(['Department Exchange Module', 'Department exchange module',
       'Political Science Exchange Mod', 'POLITICAL SCIENCE EXCHANGE MOD',
       'Global Studies Exchange Dummy', 'Topics in Business Analytics',
       'Exchange UEM', 'Special Exchange Module',
       'TOPICS IN MKTG:SEL TOPICS 2', 'Managing Change',
       'Topics in Innovation & Entrepr', 'Geo Design',
       'Exchange Elective 1', 'Exchange Elective 2',
       'Exchange ULR Breadth', 'Exchange Elective',
       'Exchange Technical Elective', 'Technical Elective',
       'Exchange UEM 6', 'Exchange Elective 10', 'UEM Exchange Module',
       'Exchange UEM 5', 'IE3 Technical Elective', 'Exchange UEM 4',
       'Exchange Elective 3', 'Exchange Elective 8',
       'Exchange UEM Module', 'Exchange UEM 7',
       'Unrestricted Elective Outside', 'Exchange UEM 3',
       'Exchange Elective 5', 'Exchange UEM 2',
       'Exchange Enrichment Module', 'Exchange Enrichment Level 1000',
       'Exchange Enrichment Level 2000',

In [36]:
module_code_title_mapping = {}
for module in jsons:
    module_code = module['moduleCode']
    module_title = module['title']
    module_code_title_mapping[module_code] = module_title
    
with open("../src/data/moduleCodeTitleMappings.json",'w') as f:
    json.dump(module_code_title_mapping, f)